## Evaluation Code

In [1]:
import torch


# Check if CUDA is available and if PyTorch is using GPU
if torch.cuda.is_available():
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")
    
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(torch.cuda.memory_summary(device=1, abbreviated=False))
    

CUDA is available. Using GPU: NVIDIA GeForce RTX 3080
cuda:1
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 1                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|  

## Load the model from checkpoint

In [7]:
from rnn_model import MNISTRNN

checkpoint_path = "models/2024-12-03_09-10-50/checkpoint_98.pth"

# Hyperparameters
image_size = 100
hidden_size = 512
num_layers = 2
num_classes = 10
batch_size = 64
learning_rate = 0.0002
num_epochs = 100
num_steps = 3  # RNN steps per image
num_kernels = 12*12

model = MNISTRNN(image_size, hidden_size, num_layers, num_classes, num_kernels, device)
model.to(device)

# Load the checkpoint
checkpoint = torch.load(checkpoint_path)

# Load the state dictionary into the model
model.load_state_dict(checkpoint['model_state_dict'])

/tmp/ipykernel_13811/953356630.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


<All keys matched successfully>

## Get the test dataloader

In [8]:
from dataloaders import get_dataloaders

train_loader, test_loader = get_dataloaders()
# Example: Iterate through the DataLoader
for images, labels in train_loader:
    print("Training batch of images shape:", images.shape)  # (batch_size, 1, 100, 100)
    print("Training batch of labels shape:", labels.shape)  # (batch_size,)
    break

for images, labels in test_loader:
    print("Test batch of images shape:", images.shape)  # (batch_size, 1, 100, 100)
    print("Test batch of labels shape:", labels.shape)  # (batch_size,)
    break

print(f"train_size: {len(train_loader)} batches, test_size: {len(test_loader)} batches")

Training batch of images shape: torch.Size([64, 1, 100, 100])
Training batch of labels shape: torch.Size([64])
Test batch of images shape: torch.Size([64, 1, 100, 100])
Test batch of labels shape: torch.Size([64])
train_size: 844 batches, test_size: 94 batches


## Evaluate the Model

In [9]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        batch_size = images.size(0)
        h0 = torch.zeros(num_layers, batch_size, hidden_size).to(images.device)
        next_actions = torch.zeros((batch_size, 2), device=images.device)
        
        for step in range(num_steps):
            class_pred, action_pred, h0, sensor_readings = model(images, next_actions, h0)
            
            next_actions = action_pred

        # print(class_pred.data)
        _, predicted = torch.max(class_pred.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # if total > 10000:
        #     break
        

    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

Accuracy of the model on the test images: 86.02%
